In [8]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import CoolProp

In [9]:
# Input widgets
a = widgets.FloatText(
    value=1.0,
    description='Not',
)

#x=CoolProp.CoolProp.PropsSI("D", "T|liquid", 400, "P", 20e6, "Water")
x=0

b = widgets.FloatText(
    value=x,
    description='Available',
)

# Button
run_button = widgets.Button(
    description='Try later',
    disabled=True,
    button_style='primary'
)

# Output area
output = widgets.Output()


In [10]:
def run_calculation(button):
    with output:
        output.clear_output()

        # Read input values
        A = a.value
        B = b.value

        # Example calculation
        x = np.linspace(0, 10, 100)
        y = A * np.sin(B * x)

        # Plot
        plt.figure(figsize=(6, 4))
        plt.plot(x, y)
        plt.title(f"A={A}, B={B}")
        plt.grid(True)
        plt.show()


In [11]:
run_button.on_click(run_calculation)

In [12]:
display(
    widgets.VBox([
        a,
        b,
        run_button,
        output
    ])
)